In [2]:
import numpy as np
import pandas as pd
from skopt import gp_minimize, gbrt_minimize 
from skopt.plots import plot_convergence, plot_objective
from rl4greencrab.envs.green_crab_monthly_env import greenCrabMonthEnv
from rl4greencrab.envs.green_crab_monthly_env_norm import greenCrabMonthEnvNormalized
from rl4greencrab import evaluate_agent, multiConstAction, simulator
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, FloatSlider

In [12]:
config = {
    "Tmax" : 200
}

In [13]:
env = greenCrabMonthEnv(config)

In [14]:
def environment_simulation(env, agent, reps=1, obs_names = None, acts_names = None):
    num_obs = np.prod(env.observation_space.shape)
    num_acts = np.prod(env.action_space.shape)
    obs_names = obs_names or [f'obs{i}' for i in range(num_obs)]
    acts_names = acts_names or [f'act{i}' for i in range(num_acts)]
    #
    data = {
        't': [],
        **{obsn: [] for obsn in obs_names},
        **{actn: [] for actn in acts_names},
        'rew': [],
        'rep': [],
        'crab_pop':[]
    }
    env = env
    agent = agent
    for rep in range(reps): # try score as average of 100 replicates, still a noisy measure
        episode_reward = 0.0
        observation, _ = env.reset()
        for t in range(env.Tmax):
            action, _ = agent.predict(observation, deterministic=True)
            #
            data['rew'].append(episode_reward)
            data['rep'].append(rep)
            data['t'].append(t)
            data['crab_pop'].append(env.state)
            for idx, obs_name in enumerate(obs_names):
                data[obs_name].append(observation[idx])
            for idx, act_name in enumerate(acts_names):
                data[act_name].append(action[idx])
            #
            observation, reward, terminated, done, info = env.step(action)
            episode_reward += reward
            #
            if terminated or done:
                break
    return data

In [15]:
agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
constantData = environment_simulation(env, agent)
constantDataDF = pd.DataFrame(constantData)
constantDataDF.head()

,t,obs0,act0,act1,act2,rew,rep,crab_pop
0,0,0.0,0.0,0.0,0.0,0.000000,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,0.0,0.0,0.0,0.0,-0.000061,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,0.0,0.0,0.0,0.0,-0.000123,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,0.0,0.0,0.0,0.0,-0.000184,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,0.0,0.0,0.0,0.0,-0.000245,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [16]:
crab_pop_df = pd.DataFrame(constantDataDF[constantDataDF['rep']==0]['crab_pop'].tolist(), columns=[f'crab_pop_{i}' for i in range(len(constantDataDF['crab_pop'][0]))])

In [17]:
new_column_names = [i for i in range(len(crab_pop_df.columns))]
crab_pop_df.columns = new_column_names

In [21]:
def change_config(w_mort, nmort, growth_k):
    config = {
        "w_mort_scale" : w_mort,
        "nmortality" : nmort,
        "growth_k": growth_k,
        "Tmax": 200
    }
    env = greenCrabMonthEnv(config)
    agent = multiConstAction(env=env, action=np.array([0.0, 0, 0.0]))
    constantData = environment_simulation(env, agent)
    constantDataDF = pd.DataFrame(constantData)
    crab_pop_df = pd.DataFrame(constantDataDF[constantDataDF['rep']==0]['crab_pop'].tolist(), 
                               columns=[f'crab_pop_{i}' for i in range(len(constantDataDF['crab_pop'][0]))])
    new_column_names = [i for i in range(len(crab_pop_df.columns))]
    crab_pop_df.columns = new_column_names
    return crab_pop_df

In [22]:
# Interactive plot function
def plot_size_distribution(time_step, w_mort_scale, nmort, growth_k):
    crab_pop_df = change_config(w_mort_scale, nmort, growth_k)
    plt.figure(figsize=(8, 5))
    plt.bar(crab_pop_df.columns, crab_pop_df.loc[time_step], color='blue', alpha=0.7)
    plt.title(f'Crab Size Distribution at {time_step}')
    plt.xlabel('Crab Size')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [24]:
interact(plot_size_distribution, time_step=crab_pop_df.index, 
         w_mort_scale=FloatSlider(min=0, max=2000, step=50, value=600), 
         nmort = FloatSlider(min=0, max=1, step=0.01, value=0.03),
        growth_k = FloatSlider(min=0, max=1, step=0.01, value=0.7),);

interactive(children=(Dropdown(description='time_step', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,…